In [21]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-bd5d  GCE       4                                             RUNNING  us-central1-a


In [22]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [23]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  6 13:09 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [25]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [26]:
# spark.stop()


In [27]:
conf = SparkConf().setAppName("pagerank")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)


24/03/06 15:40:01 INFO SparkEnv: Registering MapOutputTracker
24/03/06 15:40:01 INFO SparkEnv: Registering BlockManagerMaster
24/03/06 15:40:01 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/03/06 15:40:01 INFO SparkEnv: Registering OutputCommitCoordinator


In [28]:
spark

In [29]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '208564682pagerank' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [30]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [31]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [32]:
parquetFile = spark.read.parquet(*paths)
links = parquetFile.select("id","anchor_text")

In [33]:
# Count number of wiki pages
links.count()

6348910

Page Rank

In [34]:
def generate_graph(pages):
    ''' Compute the directed graph generated by wiki links.
    Parameters:
    -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
    Returns:
    --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
    '''
    def generate_graph(pages):
        genral = lambda item: item
        lev_pages = pages.flatMapValues(genral)
        edge_func = lambda item : (item[0], item[1]["id"])
        edges = lev_pages.map(edge_func).distinct()
        vert_func = lambda item:(item,)
        vertices = edges.flatMap(genral).distinct().map(vert_func)
        return edges, vertices

In [35]:
t_start = time()
# pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
pages_links =links.rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

24/03/06 15:44:47 WARN YarnAllocator: Container from a bad node: container_1709730490612_0015_01_000001 on host: cluster-bd5d-w-2.c.proven-reality-390308.internal. Exit status: 143. Diagnostics: [2024-03-06 15:44:46.892]Container killed on request. Exit code is 143
[2024-03-06 15:44:46.892]Container exited with a non-zero exit code 143. 
[2024-03-06 15:44:46.893]Killed by external signal
.
24/03/06 15:44:47 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 1 for reason Container from a bad node: container_1709730490612_0015_01_000001 on host: cluster-bd5d-w-2.c.proven-reality-390308.internal. Exit status: 143. Diagnostics: [2024-03-06 15:44:46.892]Container killed on request. Exit code is 143
[2024-03-06 15:44:46.892]Container exited with a non-zero exit code 143. 
[2024-03-06 15:44:46.893]Killed by external signal
.
24/03/06 15:44:47 ERROR YarnScheduler: Lost executor 1 on cluster-bd5d-w-2.c.proven-reality-390308.internal: Container from a bad node:

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9913.728782160772|
|  10568| 5385.349263642037|
|  32927|5282.0815757652745|
|  30680| 5128.233709604122|
|5843419| 4957.567686263869|
|  68253| 4769.278265355159|
|  31717| 4486.350180548309|
|  11867|  4146.41465091277|
|  14533| 3996.466440885506|
| 645042|3531.6270898037424|
|  17867| 3246.098390604141|
|5042916| 2991.945739166179|
|4689264| 2982.324883041747|
|  14532|2934.7468292031695|
|  25391| 2903.546223513399|
|   5405|2891.4163291546365|
|4764461|2834.3669873326617|
|  15573| 2783.865118158839|
|   9316|2782.0396464137693|
|8569916| 2775.286191840016|
+-------+------------------+
only showing top 20 rows



In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the
# month of August 2021
import pickle
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path)
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB)
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly
# total number of page views (5). Then, remove lines with article id or page
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

--2024-03-06 17:14:28--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.74MB/s    in 8m 33s  

2024-03-06 17:23:01 (4.65 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [ ]:
name = "pageview"
with open(f"{name}.pkl", "wb") as f:
    pickle.dump(wid2pv, f)

client = storage.Client()
bucket = client.bucket(bucket_name)
blob_posting_locs = bucket.blob(f"page_view/{name}.pkl")
blob_posting_locs.upload_from_filename(f"{name}.pkl")
